# 1. Read Web Scrapped Data from Text File and Divide it into chunks


**Text File :**
*   This WebPage : https://lhr.nu.edu.pk/fsc/programsHome/  contains info. about all the programs of FAST School of ComputingWebPage

* Data is scrapped from all the programs and data of each program is saved in separate text file

*   Each text file is further divided by  line sperator separating different info about that program

**Chunking :**


*   Each program data is further divided into 4 to 5 parts and these parts sperated by a line sperator extract these parts based on the splitter and each part is a chunk

*   Tag each chunk with the name of that department which is also the name of text file from which it is being extracted









In [ ]:
import os
# Directory containing  text files
directory = "/content/data"
import pickle
# Placeholder for the chunks
chunks = []

# Loop through each file in the directory
for filename in os.listdir(directory):
    if filename.endswith(".txt"):  # Ensure we are processing only text files
        filepath = os.path.join(directory, filename)

        # Open and read the file

        with open(filepath, 'r') as file:
            content = file.read()

        # Split the content based on the splitter
        parts = content.split("--------------------------------------------------------------------------------")  # Replace "SPLITTER" with your actual splitter

        # Tag each part(chunk) with the name of the text file (or department name)
        for part in parts:
            chunk = f"{filename} {part.strip()}"
            chunks.append(chunk.lower())

# Save text chunks to a file using pickle
with open('chunks.pkl', 'wb') as f:
    pickle.dump(chunks, f)


# Chunks Embedding using voyageai

*   Voyageai is open source embedding model

*  Voyageai api is used to send chunks in api request and in reponse it returns the embedings of these chunks

*   Voyageai api has limitations it only allows 3 requests and total of 10000 tokens to send per minute
*   So if 10000 tokens are sent in 1 request wait for one minute and then send other 10000 tokens

*   So send chunks that contains less a total of 10000 tokens in one request and wait for one minute and then send the other chunks









In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('voyageai/voyage-large-2-instruct')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [ ]:
# Divide all the chunks in group of chunks where each group contains 10000 tokens

token_limit = 9500
divided_chunks = []
temp_chunks = []
token_count = 0
for i in range(len(chunks)):
  total_tokens = len(tokenizer.tokenize(chunks[i]))
  token_count += total_tokens
  if token_count < token_limit:
      temp_chunks.append(chunks[i])
  else :
    divided_chunks.append(temp_chunks)
    temp_chunks = []
    temp_chunks.append(chunks[i])
    token_count = total_tokens
divided_chunks.append(temp_chunks)

In [ ]:
len(divided_chunks)

2

In [3]:
!pip -q install  voyageai

In [ ]:
import voyageai
import time
voyageai.api_key = 'pa-lkjkljjhhlkjkljiohiojj'
vo = voyageai.Client()
embeddings = []
for i in range(len(divided_chunks)):
  output = vo.embed(divided_chunks[i], model="voyage-large-2-instruct",input_type='document')
  embeddings.append(output.embeddings)
  if i < (len(divided_chunks) -1) :
   # Waiting for one minute
   time.sleep(60)


In [ ]:
import numpy as np
new_embeds = []
for i in range(len(embeddings)):
  for j in range(len(embeddings[i])):
       new_embeds.append(embeddings[i][j])
new_embeds = np.array(new_embeds)
del embeddings
del divided_chunks

In [ ]:
len(new_embeds)

32

In [ ]:
np.save('embeddings.npy', new_embeds)

In [4]:
%pip -q install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 13.7 MB/s eta 0:00:00


# Similarity Search using faiss


Run this Cell and provide a query about Programs in FAST School of Computing

In [ ]:
import faiss
query = input("ask Query : ")
# Function to embed user query
def embed_query(query):
    query_embedding = vo.embed([query.lower()], model="voyage-large-2-instruct")
    return np.array(query_embedding.embeddings)
embedded_query = embed_query(query)

# Create a FAISS index
embedding_dimension = new_embeds.shape[1]
index = faiss.IndexFlatL2(embedding_dimension)
index.add(new_embeds)

k = 5  # Number of nearest neighbors to search for
D, I = index.search(embedded_query, k)


# Output the results
print("Indices of nearest neighbors:", I)
print("Distances to nearest neighbors:",D)

ask Query : Give the list of courses of CS 7th semester
Indices of nearest neighbors: [[20 29  6 28 11]]
Distances to nearest neighbors: [[0.49339908 0.5538664  0.57634044 0.65144575 0.6573378 ]]


In [ ]:
prompt = "Given the context : \n"

# Prepare Prompt and send to LLM model to get response

In [ ]:
k = 5  # Number of nearest neighbors
for i in range(k):
    nearest_index = I[0][i]  # Assuming you're interested in the first set of indices (for the first query result)
    nearest_text = chunks[nearest_index]  # Assuming cleaned_list contains your text data
    # print(f"{i} Name {nearest_text}")
    prompt = prompt + nearest_text



In [ ]:
prompt = prompt + "\n\nAnswer this Question in an intelligent, concise and in a better format: "+query

In [ ]:
import google.generativeai as genai
genai.configure(api_key='AIjkhjjkhjkhkhjk')


model = genai.GenerativeModel(model_name="gemini-1.5-flash")
response = model.generate_content(prompt)
print(response.text)

## CS 7th Semester Courses:

| Sr. No. | Course Name | Credit Hours |
|---|---|---|
| 1 | CS4091 Final Year Project - I | 0+3 |
| 2 | CS3002 Information Security | 3+0 |
| 3 | CSxxxx CS Elective - IV | 3+0 |
| 4 | CS4001 Professional Practices | 3+0 |
| 5 | CS4031 Compiler Construction | 3+0 |
| 6 | CS4081 Parallel and Distributed Computing | 3+0 |

**Total:** 15 + 3 



# Complete Code in One Cell

In [8]:
import google.generativeai as genai
import faiss
import voyageai
import numpy as np
import pickle
voyageai.api_key = 'pa-kjhjkhjkhjkhjk'    # get your api key from  https://dash.voyageai.com/
vo = voyageai.Client()
genai.configure(api_key='AIlkjkljkljkljklj')     # get your api key from https://aistudio.google.com/app/apikey
model = genai.GenerativeModel(model_name="gemini-1.5-flash")


def embed_query(query):
    query_embedding = vo.embed([query.lower()], model="voyage-large-2-instruct")
    return np.array(query_embedding.embeddings)

def get_response(query,embeddings,chunks):
  prompt = "Given the context : \n"
  # Function to embed user query
  embedded_query = embed_query(query)
  # Create a FAISS index
  embedding_dimension = embeddings.shape[1]
  index = faiss.IndexFlatL2(embedding_dimension)
  index.add(embeddings)

  k = 10  # Number of nearest neighbors to search for
  D, I = index.search(embedded_query, k)
  for i in range(k):
    nearest_index = I[0][i]
    nearest_text = chunks[nearest_index]
    prompt = prompt + nearest_text
  prompt = prompt + "\n\nAnswer this Question in an intelligent, concise way and a better format based on the above provided FAST School of Computing data : "+query

  response = model.generate_content(prompt)
  return response.text

# Function to load embeddings and text chunks from files
def load_data():
    embeddings = np.load('embeddings.npy')
    with open('chunks.pkl', 'rb') as f:
        chunks = pickle.load(f)
    return embeddings, chunks

if __name__ == "__main__":
  # Load embeddings and chunks
  embeddings, chunks = load_data()
  query = input("ask Query : ")
  response = get_response(query,embeddings,chunks)
  print(response)



ask Query : BS CS 7th Semester Credit Hours
## BS CS 7th Semester Credit Hours:

| Sr. No. | Course Name | Credit Hours |
|---|---|---|
| 1 | CS4091 Final Year Project - I | 0+3 |
| 2 | CS3002 Information Security | 3+0 |
| 3 | CSxxxx CS Elective - IV | 3+0 |
| 4 | CS4001 Professional Practices | 3+0 |
| 5 | CS4031 Compiler Construction | 3+0 |
| 6 | CS4081 Parallel and Distributed Computing | 3+0 | 
| **Total** |  | **15 + 3** | 

